The exoGravity package
======================

Installation
------------

### Requirements

-   Tested on Linux/Debian 8

-   Python 2.7 or Python 3.x, ideally with x\>=5 (required for
    ruamel.yaml)

-   Astropy (\>=2.0.2)

-   ruamel.yaml \>=0.16.5 (recommended) or alternatively pyyaml \>=3.11

-   NumPy (\>=1.16.5), Scipy (\>=0.19.0)

-   cleanGravity (see installation procedure below)

### Installation procedure

To be improved

Download cleanGravity, add to path, then download exoGravity, makes
scripts executable (`chmod +x`), add to path. Test it. You’re done!

Usage
-----

The exoGravity package provides 3 scripts, which should be used
sequentially to reduce the data. The scripts are the following:

-   `astrometry_reduce.py`: is used to extract the astrometry from the
    observations. The script will output the best astrometry on the
    terminal, and will add the astrometric solution for each `planet_oi`
    in the YAML configuration file.

-   `spectrum_reduce.py`: is used to extract the spectrum from the
    observations. The script will create a “contrast.txt” and
    “covariance.txt”, to store the final spectrum and its error bars.
    Ideally, an astrometric solution should be provided for each planet
    observation in the YAML configuration file. If not, the script will
    assume the fiber to be perfectly centered on the planet, which may
    lead to bad results.

-   `swap_reduce.py` is only used for off-axis observations, for which
    the astrometry of the reference binary is not known with enough
    precision. This script will extract he astrometric solution for the
    binary from the observations themselves.

These scripts use a common YAML configuration file, which describes the
data files, the reduction parameters, etc. The exoGravity package comes
with a tool to help you create a properly formatted YAML file:
`create_config.py`. This script takes at least one argument, which is
the path to the directory in which the astrored files are stored. It
goes through the files, identifies on-planet, on-star, and swap
observations, and created an associated config file. lease see the
documentation for further options.

To use these scripts, start by putting all the “astrored” files
corresponding to your observation (i.e. on-planet, on-star, possibly
swap reference) in a common directory. To test the scripts, you can use
one of the directory contained in the `examples` directory of this
tutorial.

Instead of creating a configuration file from scratch yourself, it is
strongly advised to use the script provided, and to tweak the parameters
afterwards. Run:

``` bash
create_config.py datadir=./examples/betaPictoris_2018/ output=betaPictoris2018.yml
```

This will create the YAML file. You can now open it with a text editor.
For this demo, we will change a few parameters to accelerate the
reduction. First, in the `general` section, look for the `gofast`
parameter, and set it to `true`. Then, search for the number of points
in the RA/DEC maps (`n_ra` and `n_dec`, and set them to 50 instead of
the default 100). Note that these changes could also have been requested
at the creation of the file, by calling:

``` bash
create_config.py datadir=./examples/betaPictoris_2018/ output=betaPictoris2018.yml gofast=True nra=50 ndec=50
```

The last change we will make is to skip the reduction of some of the
files. The YAML format supports commenting (with the char `#`). In the
`general` section of the configuration file, look for the `reduce`
keyword. This gives a list (one element per line, preceded by a dash, as
per YAML specification) of keys to the planet files that will be
reduced. The keys refer to the elements of the `planet_ois` section of
this same configuration file. For this example, we will only keep the
first 5 files.

The `general` section of you config file should now look something like:

    general:
      contrast_file: null
      datadir: /data1/gravity/BetaPictoris/2018-09-22/reduced/
      declim:
      - 117.47058823529412
      - 127.47058823529412
      gofast: true
      n_dec: 50
      n_opd: 100
      n_ra:50
      noinv: false
      phaseref_mode: DF_STAR
      ralim:
      - 65.0
      - 75.0
      reduce:
      - p0
      - p1
      - p2
      - p3
      - p4
      - p5
    #  - p6
    #  - p7
    #  - p8
    #  - p9
    #  - p10
    #  - p11
    #  - p12
    #  - p13
    #  - p14
    #  - p15
    #  - p16
      save_fig: true
      star_diameter: 0.0
      star_order: 3

``` bash
astrometry_reduce.py config_file=betaPictoris2018.yml
```

At the end, the script will output the RA/DEC solution, in mas:
(RA=68.27, DEC=126.45). The script will also have added the astrometric
solution for each of the planet files in the configuration file. If you
reopen the file in a text editor, you should see the in the `planet_ois`
section, each element which was not commented in the `reduce` list now
has an `astrometric_solution`. Note that if you are using exoGravity
with the pyyaml library for managing the YAML configuration files (see
installation section), the lines commented in the YAML file do not
survive read/write cycles. Thus, is using pyyaml, the `reduce` list in
`general` section is now shorter.

Now that each planet files has an astrometric solution, you can move to
the spectrum extaction:

``` bash
spectrum_reduce.py config_file=betaPictoris2018.yml outputdir=./result
```

When the script is finished, you will have two files in the “result”
dir. You can plot the spectrum with any tool you want. For example:

``` python
  import matplotlib.pyplot as plt
  import numpy as np
  data = np.loadtxt("./result/contrast.txt", skiprows=2)
  plt.figure()
  plt.plot(data[:, 0], data[:, 1])
  plt.title("Contrast spectrum for beta Pictoris b")
  plt.xlabel("Wavelength (um)")
  plt.ylabel("Contrast")  
  plt.show()
```

Configuration File
------------------

The reduction done by the exoGravity package is parameterized using a
YAML configuration file. This configuration file contains 3 or 4 main
levels, depending on the type of observation, defined below.

### General

The `general` section contains parameters which are not file-specific,
and which are used throughout the entire reduction.

To be written